# Create train, test, val Dataset

### (Was adapted for whole Telegram Corpus)

In [ ]:
import pandas as pd
import helpers

In [ ]:
import helpers

In [ ]:
config = yaml.safe_load(open('config.yaml'))

In [ ]:
SAMPLING_SEED = config['seeds']['sampling_seed']

## Load raw data

In [ ]:
path_to_data = ""

In [ ]:
data = pd.read_csv("path_to_data", index_col=0)

In [ ]:
data.conspiracy.value_counts()

In [ ]:
# filter for source
df = data.loc[data.source=='telegram']
print('Length of selected source:', len(df))

## Stance and Sub-label Filtering

In [ ]:
df = df.loc[df.disbelief!='Yes'] # remove disbelief 
df = df.loc[((df.belief=='Yes') | (df.authenticating=='Yes') | (df.directive=='Yes') | (df['rhetorical question']=='Yes')) | (df.conspiracy=='No')] # keep only records with no ct or at least one of mentioned ct sub stance 
print('Length after stance selection:', len(df))
# reference
df = df.loc[~((df.reference=='Yes') & (df.actor=='No') & (df.strategy=='No') & (df.goal=='No'))] # remove references if the only sublabel
print('Length after reference removal:', len(df))

In [ ]:
df['conspiracy'] = df['conspiracy'].map({'Yes':1, 'No': 0})
df.rename(columns={'conspiracy': "label"}, inplace=True)

In [ ]:
df = df[~df['no_footer_no_replacement_words'].isna()]

In [ ]:
rel_columns = ['text', 'message_id', 
       'label', 'pandemic reference', 'no_footer',
       'no_footer_no_replacement_words', 'no_replacement_words',
       'no_footer_no_emoji', 'no_footer_no_replacement_words_no_emoji',
       'no_replacement_words_no_emoji']

In [ ]:
df = df[rel_columns]

In [ ]:
df.rename(columns={"text": "original_text"}, inplace=True)

## Train-Test-Split

In [ ]:
# split data into train, test & val

train = df.sample(frac = 0.8, random_state=SAMPLING_SEED)
val = df.drop(train.index).sample(frac=0.5, random_state=SAMPLING_SEED)
test = df.drop((train.index|val.index))

print(len(train), len(val), len(test))

In [ ]:
#train.to_pickle("data/train.pkl")  
#test.to_pickle("data/test.pkl")  
#val.to_pickle("data/val.pkl")  